In [1]:
try:
    sc.stop()
except:
    pass

In [2]:
from pyspark import SparkContext
from bigdl.util.common import *


# create sparkcontext with bigdl configuration
sc = SparkContext.getOrCreate(conf=create_spark_conf().setMaster("local[*]"))

Prepending /home/dyllanjr/anaconda3/envs/solarenv/lib/python3.6/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession(sparkContext = sc)

In [120]:
spark.conf.set("spark.sql.session.timeZone", "UTC")

In [121]:
#Gather Arkansas Dataset for cleansing
years = list(range(1998, 2018))
for ix, year in enumerate(years):
    if ix == 0:
        df = spark.read.csv(f"hdfs://localhost:9000/solar_data/raw/Arkansas_{year}.csv", inferSchema=True, header=True)
    else:
        to_append = spark.read.csv(f"hdfs://localhost:9000/solar_data/raw/Arkansas_{year}.csv", inferSchema=True, header=True)
        df = df.union(to_append)

In [122]:
df.show(5)

+-------------------+----+-----+---+----+------+----------+---------+---------+------------------+--------------+-----------+------------------+--------------+---+
|                _c0|Year|Month|Day|Hour|Minute|Cloud Type|Dew Point|Fill Flag|        Wind Speed|Surface Albedo|Temperature|Solar Zenith Angle|Wind Direction|GHI|
+-------------------+----+-----+---+----+------+----------+---------+---------+------------------+--------------+-----------+------------------+--------------+---+
|1998-01-01 05:00:00|1998|    1|  1|   0|     0|         0|       -5|        0|0.6000000000000001|         0.114|          3|             101.6|         182.9|  0|
|1998-01-01 05:30:00|1998|    1|  1|   0|    30|         0|       -5|        0|               0.8|         0.114|          2|            107.41|         182.9|  0|
|1998-01-01 06:00:00|1998|    1|  1|   1|     0|         0|       -5|        0|               1.1|         0.114|          2|            113.34|         172.5|  0|
|1998-01-01 06:3

In [123]:
df.count()

350400

In [124]:
df = df.withColumnRenamed('_c0', 'timestamp')

In [125]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- Cloud Type: integer (nullable = true)
 |-- Dew Point: integer (nullable = true)
 |-- Fill Flag: integer (nullable = true)
 |-- Wind Speed: double (nullable = true)
 |-- Surface Albedo: double (nullable = true)
 |-- Temperature: integer (nullable = true)
 |-- Solar Zenith Angle: double (nullable = true)
 |-- Wind Direction: double (nullable = true)
 |-- GHI: integer (nullable = true)



In [126]:
df.select('Fill Flag').groupBy('Fill Flag').count().show()

+---------+------+
|Fill Flag| count|
+---------+------+
|        1|  9936|
|        3|   589|
|        4| 15525|
|        2|     3|
|        0|324347|
+---------+------+



### Fill Flag Meaning:


0: NaN (no flags raise)   
1: Missing Image 	
2: Low Irradiance 	
3: Exceeds Clearsky 	
4: Missing CLoud Properties (does not mean cloud type is Nan as shown below)  
5: Rayleigh Violation

In [127]:
df.select('Cloud Type').groupBy('Cloud Type').count().show()

+----------+------+
|Cloud Type| count|
+----------+------+
|         1| 28995|
|         6| 19685|
|         3| 29608|
|         9|  1557|
|         4| 36421|
|         8| 24064|
|         7| 50712|
|        10|     6|
|         2|  5680|
|         0|153672|
+----------+------+



### Cloud Type Meaning:

-15: NaN  
0: Clear   
1: Probably Clear 
2: Fog 
3: Water  
4: Super-Cooled Water 
5: Mixed 
6: Opaque Ice  
7: Cirrus  
8: Overlapping  
9: Overshooting  
10: Unknown 	 
11: Dust  
12: Smoke

In [128]:
#40 unique duplicate entries!
df.groupby('timestamp').count().where('count > 1').sort('count', ascending=False).sort('timestamp').show(40)

+-------------------+-----+
|          timestamp|count|
+-------------------+-----+
|1998-04-05 07:00:00|    2|
|1998-04-05 07:30:00|    2|
|1999-04-04 07:00:00|    2|
|1999-04-04 07:30:00|    2|
|2000-04-02 07:00:00|    2|
|2000-04-02 07:30:00|    2|
|2001-04-01 07:00:00|    2|
|2001-04-01 07:30:00|    2|
|2002-04-07 07:00:00|    2|
|2002-04-07 07:30:00|    2|
|2003-04-06 07:00:00|    2|
|2003-04-06 07:30:00|    2|
|2004-04-04 07:00:00|    2|
|2004-04-04 07:30:00|    2|
|2005-04-03 07:00:00|    2|
|2005-04-03 07:30:00|    2|
|2006-04-02 07:00:00|    2|
|2006-04-02 07:30:00|    2|
|2007-03-11 07:00:00|    2|
|2007-03-11 07:30:00|    2|
|2008-03-09 07:00:00|    2|
|2008-03-09 07:30:00|    2|
|2009-03-08 07:00:00|    2|
|2009-03-08 07:30:00|    2|
|2010-03-14 07:00:00|    2|
|2010-03-14 07:30:00|    2|
|2011-03-13 07:00:00|    2|
|2011-03-13 07:30:00|    2|
|2012-03-11 07:00:00|    2|
|2012-03-11 07:30:00|    2|
|2013-03-10 07:00:00|    2|
|2013-03-10 07:30:00|    2|
|2014-03-09 07:00:00

In [129]:
len(years)
#the number of duplicates = 4 * len(years)

20

In [130]:
df.where('timestamp == "1998-04-05 03:00:00"').show()
#The hour column doesn't match the timestamp

+-------------------+----+-----+---+----+------+----------+---------+---------+----------+--------------+-----------+------------------+--------------+---+
|          timestamp|Year|Month|Day|Hour|Minute|Cloud Type|Dew Point|Fill Flag|Wind Speed|Surface Albedo|Temperature|Solar Zenith Angle|Wind Direction|GHI|
+-------------------+----+-----+---+----+------+----------+---------+---------+----------+--------------+-----------+------------------+--------------+---+
|1998-04-05 03:00:00|1998|    4|  4|  22|     0|         4|        4|        0|       2.5|         0.183|         13|             60.84|          22.1|139|
+-------------------+----+-----+---+----+------+----------+---------+---------+----------+--------------+-----------+------------------+--------------+---+



In [131]:
df.where('timestamp == "2014-03-09 03:00:00"').show()
#The hour column doesn't match the timestamp

+-------------------+----+-----+---+----+------+----------+---------+---------+----------+--------------+-----------+------------------+--------------+---+
|          timestamp|Year|Month|Day|Hour|Minute|Cloud Type|Dew Point|Fill Flag|Wind Speed|Surface Albedo|Temperature|Solar Zenith Angle|Wind Direction|GHI|
+-------------------+----+-----+---+----+------+----------+---------+---------+----------+--------------+-----------+------------------+--------------+---+
|2014-03-09 03:00:00|2014|    3|  8|  22|     0|         7|       10|        0|       0.8|         0.166|         16|             66.09|         236.1|320|
+-------------------+----+-----+---+----+------+----------+---------+---------+----------+--------------+-----------+------------------+--------------+---+



In [132]:
#Proof that each 'duplicate' timestamp has unique hour column
tss = df.groupby('timestamp').count().where('count > 1').sort('count', ascending=False).select('timestamp')

In [133]:
tss.count()

40

In [134]:
dup_rows = df.join(tss,on='timestamp', how='right')
dup_rows.show()

+-------------------+----+-----+---+----+------+----------+---------+---------+------------------+-------------------+-----------+------------------+------------------+---+
|          timestamp|Year|Month|Day|Hour|Minute|Cloud Type|Dew Point|Fill Flag|        Wind Speed|     Surface Albedo|Temperature|Solar Zenith Angle|    Wind Direction|GHI|
+-------------------+----+-----+---+----+------+----------+---------+---------+------------------+-------------------+-----------+------------------+------------------+---+
|2009-03-08 07:30:00|2009|    3|  8|   2|    30|         7|       16|        0|3.4000000000000004|              0.149|         18|            120.06|             187.7|  0|
|2009-03-08 07:30:00|2009|    3|  8|   3|    30|         7|       16|        0|               3.9|              0.149|         18|            131.34|             189.3|  0|
|2008-03-09 07:30:00|2008|    3| 10|   2|    30|         7|        4|        0|               1.5|0.12300000000000001|          6|     

In [135]:
dup_rows.count()

80

In [136]:
from pyspark.sql.functions import col
dup_rows.groupby('timestamp', 'Hour').count().sort(col('count').desc()).show()

+-------------------+----+-----+
|          timestamp|Hour|count|
+-------------------+----+-----+
|2007-03-11 07:30:00|   3|    1|
|2011-03-13 07:00:00|   2|    1|
|2004-04-04 07:00:00|   2|    1|
|2012-03-11 07:00:00|   2|    1|
|2002-04-07 07:00:00|   3|    1|
|1998-04-05 07:30:00|   2|    1|
|2005-04-03 07:30:00|   3|    1|
|1998-04-05 07:00:00|   3|    1|
|2017-03-12 07:00:00|   2|    1|
|2017-03-12 07:30:00|   2|    1|
|2016-03-13 07:00:00|   3|    1|
|2008-03-09 07:30:00|   2|    1|
|1999-04-04 07:30:00|   3|    1|
|2008-03-09 07:30:00|   3|    1|
|2000-04-02 07:00:00|   3|    1|
|2014-03-09 07:00:00|   2|    1|
|2008-03-09 07:00:00|   3|    1|
|2016-03-13 07:30:00|   3|    1|
|2002-04-07 07:30:00|   2|    1|
|2004-04-04 07:30:00|   3|    1|
+-------------------+----+-----+
only showing top 20 rows



In [ ]:
#These hour issues are from daylight savings time!!
#This means that switching to UTC should get rid of the Null entries when trying to convert timestamps

In [142]:
#I'll update the timestamp with the uniquely identifying Year|Month|Day|Hour|Minute columns
from pyspark.sql import functions as F

df = (df.withColumn('Month', F.when(F.length(F.col('Month')) == 1, F.concat(F.lit('0'), F.col('Month'))).otherwise(F.col('Month')))
        .withColumn('Day', F.when(F.length(F.col('Day')) == 1, F.concat(F.lit('0'), F.col('Day'))).otherwise(F.col('Day')))
        .withColumn('Hour', F.when(F.length(F.col('Hour')) == 1, F.concat(F.lit('0'), F.col('Hour'))).otherwise(F.col('Hour')))
        .withColumn('Minute', F.when(F.length(F.col('Minute')) == 1, F.concat(F.lit('0'), F.col('Minute'))).otherwise(F.col('Minute')))
        .withColumn('time', F.to_timestamp(F.concat(*['Year', 'Month', 'Day', 'Hour', 'Minute']), format='yyyyMMddHHmm'))
     )

df = df.drop('timestamp')
df = df.withColumnRenamed('time', 'timestamp')

In [144]:
df.where('timestamp == "2008-03-09 07:30:00"').show()


+----+-----+---+----+------+----------+---------+---------+----------+-------------------+-----------+------------------+--------------+---+-------------------+
|Year|Month|Day|Hour|Minute|Cloud Type|Dew Point|Fill Flag|Wind Speed|     Surface Albedo|Temperature|Solar Zenith Angle|Wind Direction|GHI|          timestamp|
+----+-----+---+----+------+----------+---------+---------+----------+-------------------+-----------+------------------+--------------+---+-------------------+
|2008|   03| 09|  07|    30|         7|       -2|        0|       1.1|0.12300000000000001|         -2|             144.5|         194.5|  0|2008-03-09 07:30:00|
+----+-----+---+----+------+----------+---------+---------+----------+-------------------+-----------+------------------+--------------+---+-------------------+



In [145]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Hour: string (nullable = true)
 |-- Minute: string (nullable = true)
 |-- Cloud Type: integer (nullable = true)
 |-- Dew Point: integer (nullable = true)
 |-- Fill Flag: integer (nullable = true)
 |-- Wind Speed: double (nullable = true)
 |-- Surface Albedo: double (nullable = true)
 |-- Temperature: integer (nullable = true)
 |-- Solar Zenith Angle: double (nullable = true)
 |-- Wind Direction: double (nullable = true)
 |-- GHI: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [147]:
#No more duplicate timestamps.
df.groupby('timestamp').count().where('count > 1').sort('count', ascending=False).sort('timestamp').show()

+---------+-----+
|timestamp|count|
+---------+-----+
+---------+-----+

